# Ensemple CB recommender based on Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18703 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18703 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18703 [00:00<?, ?it/s]

In [12]:
item_index = 10

In [28]:
recommenders_by_name['genres'].recommend(item_index).show()


Recommender: genres
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,id,title,genres
10,0.000000,8360,Shrek 2,Adventure Animation Children Comedy Musical Ro...
3049,0.000017,1907,Mulan,Adventure Animation Children Comedy Drama Musi...
13958,0.000066,136373,Drake And Josh Go Hollywood,Adventure Children Comedy Crime
2792,0.000067,1566,Hercules,Adventure Animation Children Comedy Musical
2780,0.000067,709,Oliver & Company,Adventure Animation Children Comedy Musical


In [29]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [39]:
recommender_group.recommend(item_index).show()


Recommender: genres
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,id,title,genres
10,0.000000,8360,Shrek 2,Adventure Animation Children Comedy Musical Ro...
3049,0.000017,1907,Mulan,Adventure Animation Children Comedy Drama Musi...
13958,0.000066,136373,Drake And Josh Go Hollywood,Adventure Children Comedy Crime
2792,0.000067,1566,Hercules,Adventure Animation Children Comedy Musical
2780,0.000067,709,Oliver & Company,Adventure Animation Children Comedy Musical



Recommender: overview
Item


...

,distance,id,title,overview
10,0.000000,8360,Shrek 2,"Shrek, Fiona and Donkey set off to Far, Far Aw..."
10690,0.027303,113674,Proxy,While walking home from her latest OB appointm...
1640,0.028103,6093,"Last Unicorn, The","From a riddle-speaking butterfly, a unicorn le..."
9726,0.028415,173449,Till Night Do Us Part,"A genuine satiric comedy, based on real conver..."
150,0.029391,47,Seven,Two homicide detectives are on a desperate hun...



Recommender: tags
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,id,title,tags
10,0.000000,8360,Shrek 2,(s)vcd 2020-2 relook 4 andrew adamson animatio...
5665,0.010161,3271,Of Mice and Men,adapted from:book annemari author:john steinbe...
2400,0.015264,152970,Hunt for the Wilderpeople,adapted from:book author:barry crump based on ...
3,0.015906,4306,Shrek,3 adventure almost favorite andrew adamson ani...
146,0.017838,80549,Easy A,amanda bynes anti christianity awesome ladies ...


In [50]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.25, 'overview': 0.25, 'genres': 0.5}
)

In [ ]:
recommender.recommend(item_index, k=5).show()


Recommender: ['genres', 'overview', 'tags']
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,title,genres,overview,tags
0,0.000000,Shrek 2,Adventure Animation Children Comedy Musical Ro...,"Shrek, Fiona and Donkey set off to Far, Far Aw...",(s)vcd 2020-2 relook 4 andrew adamson animatio...
1,0.009187,Shrek,Adventure Animation Children Comedy Fantasy Ro...,It ain't easy bein' green -- especially if you...,3 adventure almost favorite andrew adamson ani...
2,0.014344,Aladdin,Adventure Animation Children Comedy Musical,Princess Jasmine grows tired of being forced t...,(s)vcd 2d animation action adventure aftercred...
3,0.015225,How to Train Your Dragon,Adventure Animation Children Fantasy IMAX,As the son of a Viking leader on the cusp of m...,3d 55 movies every kid should see--entertainme...
4,0.015749,Chicken Run,Animation Children Comedy,Having been hopelessly repressed and facing ev...,2 aardman aardman studios almost favorite anim...
